# [Chefkoch.de](http://www.chefkoch.de/)
------

## Ziel: 
### Scraping der Hauptrezeptesammlung von über 300'000 verschiedenen Rezepten (1.Teil)

In [44]:
import os
import time
import datetime as dt
from datetime import datetime
from multiprocessing import Pool

from time import sleep, time
from random import randint, choice
import requests
import urllib.request
from bs4 import BeautifulSoup
import csv

Daten wie Rezeptname, Bewertung, Datum vom Upload des Rezeptes, etc. werden in eine csv Datei gespeichert.
Falls das Rezept ein Bild hat, wird das Thumbnail im Ordner **search_thumbnails** abgelegt.

In [45]:
# OS
NOW           = dt.datetime.now()
FILE_NAME     = 'chefkoch_rezepte_' + NOW.strftime('%d-%m-%Y') + '.csv'
DATASET_FOLDER = 'input/csv_files/'
IMGS_FOLDER  = 'input/images/search_thumbnails/'

# Chefkoch.de Seite
CHEFKOCH_URL  = 'http://www.chefkoch.de'
START_URL     = 'http://www.chefkoch.de/rs/s'
CATEGORY      = '/Rezepte.html'

Alle 300k Rezepte sortiert nach Datum: http://www.chefkoch.de/rs/s30o3/Rezepte.html

Wenn man Website Scrapping durchführt, ist es wichtig die robots.txt Datei zu respektieren. Manche Administratoren möchten nicht das bestimmte Directories von Bots besucht werden. https://www.chefkoch.de/robots.txt liefert:

- User-agent: *  # directed to all spiders, not just Scooter
- Disallow: /cgi-bin
- Disallow: /stats
- Disallow: /pictures/fotoalben/
- Disallow: /forumuploads/
- Disallow: /pictures/user/
- Disallow: /user/
- Disallow: /avatar/
- Disallow: /cms/
- Disallow: /produkte/
- Disallow: /how2videos/

Aufgeführt sind Directories die uns gar nicht interessieren, weshalb man getrost weiter machen kann. Nichtsdestotrotz  sind Massnahmen, wie zufällige Headers und genügend grosse Pausen zwischen den einzelnen Requests empfehlenswert, um einen möglichen Ban der Website zu vermeiden.

In [46]:
desktop_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0']

def random_headers():
    return {'User-Agent': choice(desktop_agents),'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

In [47]:
category_url = START_URL + '0o3' + CATEGORY

def _get_html(url):
    page = ''
    while page == '':
        try:
            page = requests.get(url, headers=random_headers())
        except:
            print('Connection refused')
            time.sleep(10)
            continue
    return page.text

def _get_total_pages(html):
    soup = BeautifulSoup(html, 'lxml')
    total_pages = soup.find('div', class_='ck-pagination qa-pagination').find('a', class_='qa-pagination-pagelink-last').text
    return int(total_pages)

html_text_total_pages = _get_html(category_url)
total_pages = _get_total_pages(html_text_total_pages)
print('Total pages: ', total_pages)

Total pages:  10560


Liste von allen einzelnen Rezepteurls bei Chefkoch im folgenden Format:
1. Seite: http&#58;//www.chefkoch.de/rs/s**0**o3/Rezepte.html
2. Seite: http&#58;//www.chefkoch.de/rs/s**30**o3/Rezepte.html
3. Seite: http&#58;//www.chefkoch.de/rs/s**60**o3/Rezepte.html
4. Seite: ...

Auf einer Seite erhält man 30 Rezepte. Um jede Seite aufrufen zu können, muss man nur die Zahl zwischen **s** und **o3** um 30 erhöhen.

In [48]:
url_list = []

for i in range(0, total_pages + 1):
    url_to_scrap = START_URL + str(i * 30) + 'o3' + CATEGORY
    url_list.append(url_to_scrap)

In [49]:
from pprint import pprint
# Die ersten 30 Seiten:
pprint(url_list[:30])

['http://www.chefkoch.de/rs/s0o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s30o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s60o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s90o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s120o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s150o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s180o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s210o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s240o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s270o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s300o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s330o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s360o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s390o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s420o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s450o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s480o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s510o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s540o3/Rezepte.html',
 'http://www.chefkoch.de/rs/s570o3/Rezepte.html',
 'htt

In [53]:
def _write_to_recipes(data):
    path = DATASET_FOLDER + FILE_NAME
    with open(path, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow((data['recipe_id'],
                        data['recipe_name'],
                        data['average_rating'],
                        data['stars_shown'],
                        data['votes'],
                        data['difficulty'],
                        data['preparation_time'],
                        data['date'],
                        data['link'],
                        data['has_picture']))

def _get_picture_link(item):
    item_class = item.find('picture').find('img').get('class')
    if item_class == ['lazyload']:
        img_link = item.find('picture').find('img').get('data-srcset')
    else: 
        img_link = item.find('picture').find('source').get('srcset')
    return(img_link)

def _get_front_page(html):
    soup = BeautifulSoup(html, 'lxml')
    lis = soup.find_all('li', class_="search-list-item")
    
    for index, li in enumerate(lis):

        # get rezept ID
        try:
            id_r = li.get('id')
        except:
            id_r = ''

        # bild speichern falls eins verügbar
        try: 
            if li.find('picture') is not None:
                img_link = _get_picture_link(li)
                img_name = IMGS_FOLDER + str(id_r) + '.jpg'
                urllib.request.urlretrieve(img_link, img_name)
                has_pic = 'yes'
            else: 
                has_pic = 'no'
        except:
            has_pic = ''

        # link
        try:
            link = CHEFKOCH_URL + li.find('a').get('href')
        except:
            link = ''

        # name des rezeptes
        try:
            name = li.find('div', class_='search-list-item-title').text.strip()
        except:
            name = ''

        # durchschnitts bewertung von nutzern
        try:
            stars = li.find('span', class_='search-list-item-uservotes-stars').get('title')
        except:
            stars = ''

        # anzahl sterne
        try:
            stars_shown = li.find('span', class_='search-list-item-uservotes-stars').get('class')[1]
        except:
            stars_shown = ''

        # anzahl votes
        try:
            votes = li.find('span', class_='search-list-item-uservotes-count').text.strip()
        except:
            votes = ''

        # schwierigkeitsgrad des rezeptes => simpel, normal oder pfiffig
        try:
            difficulty = li.find('span', class_='search-list-item-difficulty').text.strip()

        except:
            difficulty = ''

        # zubereitungs zeit
        try:
            preptime = li.find('span', class_='search-list-item-preptime').text.strip()
        except:
            preptime = ''

        # datum
        try:
            date = li.find('span', class_='search-list-item-activationdate').text.strip()
        except:
            date = ''

        # write dictionary
        data = {'recipe_id' : id_r,
                'recipe_name' : name,
                'average_rating': stars,
                'stars_shown' : stars_shown,
                'votes' : votes,
                'difficulty' : difficulty,
                'preparation_time' : preptime,
                'has_picture' : has_pic,
                'date' : date,
                'link' : link}
        
        # append file
        _write_to_recipes(data)
        
def scrap_main(url):
    print('Current url: ', url)
    html = _get_html(url)
    _get_front_page(html)
    #sleep(randint(1, 2))

In [54]:
start_time = time()
with Pool(1) as p:
    p.map(scrap_main, url_list[7813:])
print("--- %s seconds ---" % (time() - start_time))

Current url:  http://www.chefkoch.de/rs/s234390o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234420o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234450o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234480o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234510o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234540o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234570o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234600o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234630o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234660o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234690o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234720o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234750o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234780o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234810o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s234840o3/Rezep

Current url:  http://www.chefkoch.de/rs/s238320o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238350o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238380o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238410o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238440o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238470o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238500o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238530o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238560o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238590o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238620o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238650o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238680o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238710o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238740o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s238770o3/Rezep

Current url:  http://www.chefkoch.de/rs/s242250o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242280o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242310o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242340o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242370o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242400o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242430o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242460o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242490o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242520o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242550o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242580o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242610o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242640o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242670o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s242700o3/Rezep

Current url:  http://www.chefkoch.de/rs/s246180o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246210o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246240o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246270o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246300o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246330o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246360o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246390o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246420o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246450o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246480o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246510o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246540o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246570o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246600o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s246630o3/Rezep

Current url:  http://www.chefkoch.de/rs/s250110o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250140o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250170o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250200o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250230o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250260o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250290o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250320o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250350o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250380o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250410o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250440o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250470o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250500o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250530o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s250560o3/Rezep

...

Current url:  http://www.chefkoch.de/rs/s297270o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297300o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297330o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297360o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297390o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297420o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297450o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297480o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297510o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297540o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297570o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297600o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297630o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297660o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297690o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s297720o3/Rezep

Current url:  http://www.chefkoch.de/rs/s301200o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301230o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301260o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301290o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301320o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301350o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301380o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301410o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301440o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301470o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301500o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301530o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301560o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301590o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301620o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s301650o3/Rezep

Current url:  http://www.chefkoch.de/rs/s305130o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305160o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305190o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305220o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305250o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305280o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305310o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305340o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305370o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305400o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305430o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305460o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305490o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305520o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305550o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s305580o3/Rezep

Current url:  http://www.chefkoch.de/rs/s309060o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309090o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309120o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309150o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309180o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309210o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309240o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309270o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309300o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309330o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309360o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309390o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309420o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309450o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309480o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s309510o3/Rezep

Current url:  http://www.chefkoch.de/rs/s312990o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313020o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313050o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313080o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313110o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313140o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313170o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313200o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313230o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313260o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313290o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313320o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313350o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313380o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313410o3/Rezepte.html
Current url:  http://www.chefkoch.de/rs/s313440o3/Rezep

## CSV Datei lesen und korrigieren

Unter input/csv_files/ findet man die erstellte CSV Datei.
Grösse: 62.1 MB

In [4]:
!ls input/csv_files/

chefkoch_rezepte_26-12-2017.csv


In [6]:
import pandas as pd
chef_rezepte = pd.read_csv('input/csv_files/chefkoch_rezepte_26-12-2017.csv', header=None)
chef_rezepte.head()

,0,1,2,3,4,5,6,7,8,9
0,recipe-3449791514066439,Bratapfel-Rotkohl,0 Bewertungen - Ø 2.5,star-unb,(0),normal,45 min.,24.12.2017,http://www.chefkoch.de/rezepte/344979151406643...,no
1,recipe-3449721514056932,„Frühstückswolke 7“ aus Fiefhusen,0 Bewertungen - Ø 2.5,star-unb,(0),pfiffig,15 min.,24.12.2017,http://www.chefkoch.de/rezepte/344972151405693...,no
2,recipe-3449701514054063,Deichgrafensalat von der Westküste,0 Bewertungen - Ø 2.5,star-unb,(0),normal,30 min.,24.12.2017,http://www.chefkoch.de/rezepte/344970151405406...,no
3,recipe-3449651514047893,Geschichteter Kohl,0 Bewertungen - Ø 2.5,star-unb,(0),normal,40 min.,24.12.2017,http://www.chefkoch.de/rezepte/344965151404789...,no
4,recipe-3449681514052511,Rinderlendenragout,0 Bewertungen - Ø 2.5,star-unb,(0),normal,15 min.,24.12.2017,http://www.chefkoch.de/rezepte/344968151405251...,no


In [22]:
chef_rezepte[[8]][:10] # erste 10 zeilen der 8. spalte

,8
0,http://www.chefkoch.de/rezepte/344979151406643...
1,http://www.chefkoch.de/rezepte/344972151405693...
2,http://www.chefkoch.de/rezepte/344970151405406...
3,http://www.chefkoch.de/rezepte/344965151404789...
4,http://www.chefkoch.de/rezepte/344968151405251...
5,http://www.chefkoch.de/rezepte/344948151403464...
6,http://www.chefkoch.de/rezepte/344964151404709...
7,http://www.chefkoch.de/rezepte/344957151404240...
8,http://www.chefkoch.de/rezepte/344947151403342...
9,http://www.chefkoch.de/rezepte/344924151395310...


Beim Scraping ist ein Fehler unterlaufen. Die Links müssen als https Links und nicht als http Links gespeichert werden.
### Korrektur:

In [23]:
chef_rezepte[8] = chef_rezepte[8].str.replace('http', 'https')

In [26]:
chef_rezepte[[8]][:10]

,8
0,https://www.chefkoch.de/rezepte/34497915140664...
1,https://www.chefkoch.de/rezepte/34497215140569...
2,https://www.chefkoch.de/rezepte/34497015140540...
3,https://www.chefkoch.de/rezepte/34496515140478...
4,https://www.chefkoch.de/rezepte/34496815140525...
5,https://www.chefkoch.de/rezepte/34494815140346...
6,https://www.chefkoch.de/rezepte/34496415140470...
7,https://www.chefkoch.de/rezepte/34495715140424...
8,https://www.chefkoch.de/rezepte/34494715140334...
9,https://www.chefkoch.de/rezepte/34492415139531...


In [28]:
chef_rezepte.columns = ['recipe_id', 'recipe_name', 'average_rating',
                        'stars_shown', 'votes', 'difficulty', 'preparation_time',
                        'date', 'link', 'has_picture']

In [31]:
chef_rezepte.head()

,recipe_id,recipe_name,average_rating,stars_shown,votes,difficulty,preparation_time,date,link,has_picture
0,recipe-3449791514066439,Bratapfel-Rotkohl,0 Bewertungen - Ø 2.5,star-unb,(0),normal,45 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497915140664...,no
1,recipe-3449721514056932,„Frühstückswolke 7“ aus Fiefhusen,0 Bewertungen - Ø 2.5,star-unb,(0),pfiffig,15 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497215140569...,no
2,recipe-3449701514054063,Deichgrafensalat von der Westküste,0 Bewertungen - Ø 2.5,star-unb,(0),normal,30 min.,24.12.2017,https://www.chefkoch.de/rezepte/34497015140540...,no
3,recipe-3449651514047893,Geschichteter Kohl,0 Bewertungen - Ø 2.5,star-unb,(0),normal,40 min.,24.12.2017,https://www.chefkoch.de/rezepte/34496515140478...,no
4,recipe-3449681514052511,Rinderlendenragout,0 Bewertungen - Ø 2.5,star-unb,(0),normal,15 min.,24.12.2017,https://www.chefkoch.de/rezepte/34496815140525...,no


In [32]:
chef_rezepte.to_csv('input/csv_files/chefkoch_rezepte_27-12-2017.csv')

### Umbennenung der Thumbnails: Hänge ein 0 hintendran -> Thumbnail ist das erste Bild des Rezeptes.
#### 1, 2, 3, ..., n Bilder kommen im Teil 2 dazu.

In [ ]:
import os
path = 'input/images/search_thumbnails/'
files = os.listdir(path)
i = 1
for file in files:
    filename, file_extension = os.path.splitext(file)
    os.rename(os.path.join(path, file), os.path.join(path, filename + '-0' + file_extension))
    print('renamed: ', i)
    i = i+1

# Weiter gehts mit Teil 2: 02_rezepte_details.ipynb